1.Посчитайте, сколько компаний закрылось.


In [1]:
SELECT COUNT(status)

FROM company
WHERE status = 'closed'

SyntaxError: invalid syntax (2363153154.py, line 1)

2.Отобразите количество привлечённых средств для новостных компаний США. Используйте данные из таблицы company. Отсортируйте таблицу по убыванию значений в поле funding_total .

In [ ]:
SELECT funding_total

FROM company
WHERE category_code = 'news'
AND country_code = 'USA'
ORDER BY funding_total DESC

3.Найдите общую сумму сделок по покупке одних компаний другими в долларах. Отберите сделки, которые осуществлялись только за наличные с 2011 по 2013 год включительно.

In [ ]:
SELECT SUM(price_amount)

FROM acquisition
WHERE term_code = 'cash'
AND EXTRACT(YEAR FROM CAST(acquired_at AS date)) BETWEEN 2011 AND 2013

4.Отобразите имя, фамилию и названия аккаунтов людей в твиттере, у которых названия аккаунтов начинаются на 'Silver'.

In [ ]:
SELECT first_name,
       last_name,
       twitter_username
       
FROM people
WHERE twitter_username LIKE 'Silver%'

5. Выведите на экран всю информацию о людях, у которых названия аккаунтов в твиттере содержат подстроку 'money', а фамилия начинается на 'K'.

In [ ]:
SELECT *
       
FROM people
WHERE twitter_username LIKE '%money%'
AND last_name LIKE 'K%'

6. Для каждой страны отобразите общую сумму привлечённых инвестиций, которые получили компании, зарегистрированные в этой стране. Страну, в которой зарегистрирована компания, можно определить по коду страны. Отсортируйте данные по убыванию суммы.

In [ ]:
SELECT country_code,
       SUM(funding_total)
       
FROM company
GROUP BY country_code
ORDER BY SUM(funding_total) DESC

7. Составьте таблицу, в которую войдёт дата проведения раунда, а также минимальное и максимальное значения суммы инвестиций, привлечённых в эту дату.
Оставьте в итоговой таблице только те записи, в которых минимальное значение суммы инвестиций не равно нулю и не равно максимальному значению.

In [ ]:
SELECT funded_at AS date_of_round, --DATE_TRUNC('day', CAST(fr.funded_at AS timestamp)) AS date_of_round,
       MIN(raised_amount) AS min_amount_of_investment,
       MAX(raised_amount) AS max_amount_of_investment
       
       
FROM funding_round AS fr
--FULL JOIN company AS c ON fr.company_id = c.id

GROUP BY date_of_round
HAVING MIN(raised_amount) != 0 AND  MIN(raised_amount) != MAX(raised_amount)

8.Создайте поле с категориями:
Для фондов, которые инвестируют в 100 и более компаний, назначьте категорию high_activity.
Для фондов, которые инвестируют в 20 и более компаний до 100, назначьте категорию middle_activity.
Если количество инвестируемых компаний фонда не достигает 20, назначьте категорию low_activity.
Отобразите все поля таблицы fund и новое поле с категориями.

In [ ]:
SELECT *,


    CASE 
        WHEN invested_companies >= 100 THEN 'high_activity'
        WHEN invested_companies >= 20 AND invested_companies <= 99 THEN 'middle_activity'
        WHEN invested_companies < 20 THEN 'low_activity'
    END

FROM fund

9. Для каждой из категорий, назначенных в предыдущем задании, посчитайте округлённое до ближайшего целого числа среднее количество инвестиционных раундов, в которых фонд принимал участие. Выведите на экран категории и среднее число инвестиционных раундов. Отсортируйте таблицу по возрастанию среднего.

In [ ]:
with
    ds as (
            SELECT *,
                    CASE
                        WHEN invested_companies>=100 THEN 'high_activity'
                        WHEN invested_companies>=20 THEN 'middle_activity'
                        ELSE 'low_activity'
                    END AS activity
            FROM fund)


SELECT activity, 
       round(avg(investment_rounds)) from ds
group by activity
order by round

10. Проанализируйте, в каких странах находятся фонды, которые чаще всего инвестируют в стартапы. 
Для каждой страны посчитайте минимальное, максимальное и среднее число компаний, в которые инвестировали фонды этой страны, основанные с 2010 по 2012 год включительно. Исключите страны с фондами, у которых минимальное число компаний, получивших инвестиции, равно нулю. Выгрузите десять самых активных стран-инвесторов.
Отсортируйте таблицу по среднему количеству компаний от большего к меньшему, а затем по коду страны в лексикографическом порядке.

In [ ]:
SELECT country_code,
       MIN(invested_companies),
       MAX(invested_companies),
       AVG(invested_companies)
       
FROM fund
WHERE EXTRACT(YEAR FROM founded_at) BETWEEN 2010 AND 2012
GROUP BY country_code
HAVING MIN(invested_companies)!= 0
ORDER BY AVG(invested_companies) DESC, country_code


LIMIT 10

11. Отобразите имя и фамилию всех сотрудников стартапов. Добавьте поле с названием учебного заведения, которое окончил сотрудник, если эта информация известна.

In [ ]:
SELECT p.first_name,
       p.last_name,
       e.instituition
       
FROM people AS p
LEFT JOIN education AS e ON p.id = e.person_id

12. Для каждой компании найдите количество учебных заведений, которые окончили её сотрудники. Выведите название компании и число уникальных названий учебных заведений. Составьте топ-5 компаний по количеству университетов.

SELECT c.name,
       COUNT(DISTINCT e.instituition) AS instituition
       
FROM company AS c
JOIN people AS p ON c.id = p.company_id
JOIN education AS e ON p.id = e.person_id

GROUP BY c.name
ORDER BY COUNT(DISTINCT e.instituition) DESC
LIMIT 5

13. Составьте список с уникальными названиями закрытых компаний, для которых первый раунд финансирования оказался последним.

In [ ]:
SELECT DISTINCT c.name

FROM company AS c
LEFT JOIN funding_round AS fr ON c.id = fr.company_id
WHERE c.status = 'closed' 
AND is_first_round = 1
AND is_last_round = 1;

14. Составьте список уникальных номеров сотрудников, которые работают в компаниях, отобранных в предыдущем задании.

In [ ]:

SELECT p.id
FROM company AS c
INNER JOIN funding_round AS fr ON c.id = fr.company_id
INNER JOIN people AS p ON p.company_id = c.id
WHERE c.status = 'closed' 
                   AND is_first_round = 1
                   AND is_last_round = 1
GROUP BY c.name,p.id

15. Составьте таблицу, куда войдут уникальные пары с номерами сотрудников из предыдущей задачи и учебным заведением, которое окончил сотрудник.

In [ ]:
SELECT p.id,
       e.instituition 
FROM company AS c
INNER JOIN funding_round AS fr ON c.id = fr.company_id
INNER JOIN people AS p ON p.company_id = c.id
INNER JOIN education AS e ON p.id = e.person_id
WHERE c.status = 'closed' 
                   AND is_first_round = 1
                   AND is_last_round = 1
GROUP BY c.name,p.id, e.instituition

16. Посчитайте количество учебных заведений для каждого сотрудника из предыдущего задания. При подсчёте учитывайте, что некоторые сотрудники могли окончить одно и то же заведение дважды.

In [ ]:
SELECT DISTINCT p.id,
       COUNT(e.instituition) AS education
FROM education AS e
INNER JOIN people AS p ON p.id = e.person_id
WHERE p.id IN (SELECT p.id
                      
                FROM company AS c

                INNER JOIN funding_round AS fr ON c.id = fr.company_id
                INNER JOIN people AS p ON p.company_id = c.id
               
                WHERE c.status = 'closed' 
                AND is_first_round = 1
                AND is_last_round = 1
                GROUP BY c.name, p.id)


GROUP BY p.id
ORDER BY education DESC

17. Дополните предыдущий запрос и выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники разных компаний. Нужно вывести только одну запись, группировка здесь не понадобится.

In [ ]:
SELECT AVG(education)

FROM 
    (SELECT DISTINCT p.id,
            COUNT(e.instituition) AS education
            FROM education AS e
            INNER JOIN people AS p ON p.id = e.person_id
            WHERE p.id IN (SELECT p.id
                      
                           FROM company AS c

                            INNER JOIN funding_round AS fr ON c.id = fr.company_id
                            INNER JOIN people AS p ON p.company_id = c.id
               
                            WHERE c.status = 'closed' 
                            AND is_first_round = 1
                            AND is_last_round = 1
                            GROUP BY c.name, p.id)


        GROUP BY p.id
        ORDER BY education DESC) AS number_education;

18. Напишите похожий запрос: выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники Facebook*.
*(сервис, запрещённый на территории РФ)

In [ ]:
SELECT AVG(dsa.education)

FROM
    (SELECT p.id,
            COUNT(e.instituition) AS education
            FROM company AS c
           
            INNER JOIN people AS p ON p.company_id = c.id
            INNER JOIN education AS e ON p.id = e.person_id
            WHERE c.name = 'Facebook' 
                  
            GROUP BY p.id) AS dsa;

19. Составьте таблицу из полей:
name_of_fund — название фонда;
name_of_company — название компании;
amount — сумма инвестиций, которую привлекла компания в раунде.
В таблицу войдут данные о компаниях, в истории которых было больше шести важных этапов, а раунды финансирования проходили с 2012 по 2013 год включительно.

In [ ]:
SELECT f.name AS name_of_fund,
       c.name AS name_of_company,
       fr.raised_amount AS amount 

FROM investment AS i

INNER JOIN company AS c ON i.company_id = c.id
INNER JOIN fund AS f ON i.fund_id = f.id
INNER JOIN funding_round AS fr ON i.funding_round_id = fr.id

WHERE c.milestones > 6
AND EXTRACT(YEAR FROM funded_at)  BETWEEN 2012 AND 2013

20. Выгрузите таблицу, в которой будут такие поля:
название компании-покупателя;
сумма сделки;
название компании, которую купили;
сумма инвестиций, вложенных в купленную компанию;
доля, которая отображает, во сколько раз сумма покупки превысила сумму вложенных в компанию инвестиций, округлённая до ближайшего целого числа.
Не учитывайте те сделки, в которых сумма покупки равна нулю. Если сумма инвестиций в компанию равна нулю, исключите такую компанию из таблицы. 
Отсортируйте таблицу по сумме сделки от большей к меньшей, а затем по названию купленной компании в лексикографическом порядке. Ограничьте таблицу первыми десятью записями.

In [ ]:
with
t1 AS (SELECT c.name as buyer_company,
              a.price_amount as purchase_sum,
              a.id AS key
      FROM company AS c
      right join acquisition AS a ON a.acquiring_company_id = c.id
      WHERE price_amount > 0
           
      ),

t2 AS (SELECT c.name AS bought_company,
       a.id AS key,
       c.funding_total as invested_sum
       
       FROM company AS c
       Right JOIN acquisition AS a ON c.id = a.acquired_company_id
       WHERE funding_total > 0)


SELECT t1.buyer_company,
       t1.purchase_sum,
       t2.bought_company,
       t2.invested_sum,
       round(purchase_sum / invested_sum) AS rate
FROM t1 inner join t2 ON t1.key = t2.key
ORDER BY purchase_sum DESC, bought_company ASC
LIMIT 10
       

21. Выгрузите таблицу, в которую войдут названия компаний из категории social, получившие финансирование с 2010 по 2013 год включительно. Проверьте, что сумма инвестиций не равна нулю. Выведите также номер месяца, в котором проходил раунд финансирования.

In [ ]:
SELECT c.name AS name_of_company,
       EXTRACT(MONTH FROM CAST(funded_at AS timestamp)) AS found_month

FROM company AS c
INNER JOIN funding_round AS fr ON c.id = fr.company_id
WHERE EXTRACT(YEAR FROM CAST (funded_at AS timestamp)) IN (2010, 2011, 2012, 2013)
AND fr.raised_amount > 0
AND c.category_code = 'social'

22. Отберите данные по месяцам с 2010 по 2013 год, когда проходили инвестиционные раунды. Сгруппируйте данные по номеру месяца и получите таблицу, в которой будут поля:
номер месяца, в котором проходили раунды;
количество уникальных названий фондов из США, которые инвестировали в этом месяце;
количество компаний, купленных за этот месяц;
общая сумма сделок по покупкам в этом месяце.

In [ ]:
WITH
t1 AS(SELECT EXTRACT(MONTH FROM CAST(fr.funded_at AS DATE)) AS number_of_month,
             COUNT(DISTINCT f.name) AS name_of_fund
      FROM funding_round AS fr
      INNER JOIN investment AS i ON fr.id = i.funding_round_id
      INNER JOIN fund AS f ON i.fund_id = f.id
      WHERE EXTRACT(YEAR FROM CAST(fr.funded_at AS DATE)) BETWEEN 2010 AND 2013
      AND f.country_code = 'USA'
      GROUP BY number_of_month),
      
t2 AS(SELECT EXTRACT(MONTH FROM CAST(acquired_at AS DATE)) AS  number_of_month,
             COUNT(acquired_company_id) AS bought_company,
             SUM(price_amount) AS total_bought
      FROM acquisition
      WHERE EXTRACT(YEAR FROM CAST(acquired_at AS DATE)) BETWEEN 2010 AND 2013
      GROUP BY number_of_month)
      
SELECT t1.number_of_month,
       t1.name_of_fund,
       t2.bought_company,
       t2.total_bought
       
FROM t1 INNER JOIN t2 ON t1.number_of_month = t2.number_of_month


23. Составьте сводную таблицу и выведите среднюю сумму инвестиций для стран, в которых есть стартапы, зарегистрированные в 2011, 2012 и 2013 годах. Данные за каждый год должны быть в отдельном поле. Отсортируйте таблицу по среднему значению инвестиций за 2011 год от большего к меньшему.

In [ ]:
WITH
y_2011 AS(SELECT country_code AS country,
          AVG(funding_total) AS invested_avg
          FROM company
          WHERE EXTRACT(YEAR FROM CAST(founded_at AS DATE)) = 2011
          GROUP BY country), 
          
y_2012 AS(SELECT country_code AS country,
          AVG(funding_total) AS invested_avg
          FROM company
          WHERE EXTRACT(YEAR FROM CAST(founded_at AS DATE)) = 2012
          GROUP BY country), 
          
y_2013 AS(SELECT country_code AS country,
          AVG(funding_total) AS invested_avg
          FROM company
          WHERE EXTRACT(YEAR FROM CAST(founded_at AS DATE)) = 2013
          GROUP BY country)
          
SELECT y_2011.country,
       y_2011.invested_avg as invested_2011,
       y_2012.invested_avg as invested_2012,
       y_2013.invested_avg as invested_2013
          
FROM y_2011 INNER JOIN y_2012 ON y_2011.country = y_2012.country
INNER JOIN y_2013 ON  y_2012.country = y_2013.country

--GROUP BY country
ORDER BY invested_2011 DESC